In [1]:
import os
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.linear_model import LogisticRegression, SGDClassifier

In [2]:
import sklearn
print(sklearn.__version__)

0.19.2


In [3]:
# Поменяйте на свой путь к данным
PATH_TO_DATA = 'capstone_user_identification'

In [4]:
train_df_400 = pd.read_csv(os.path.join(PATH_TO_DATA,'train_sessions_400users.csv'), 
                           index_col='session_id')

test_df_400 = pd.read_csv(os.path.join(PATH_TO_DATA,'test_sessions_400users.csv'), 
                           index_col='session_id')

In [5]:
train_df_400.head()

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,...,time6,site7,time7,site8,time8,site9,time9,site10,time10,user_id
session_id,,,,,,,,,,,,,,,,,,,,,
1,23713,2014-03-24 15:22:40,23720.0,2014-03-24 15:22:48,23713.0,2014-03-24 15:22:48,23713.0,2014-03-24 15:22:54,23720.0,2014-03-24 15:22:54,...,2014-03-24 15:22:55,23713.0,2014-03-24 15:23:01,23713.0,2014-03-24 15:23:03,23713.0,2014-03-24 15:23:04,23713.0,2014-03-24 15:23:05,653
2,8726,2014-04-17 14:25:58,8725.0,2014-04-17 14:25:59,665.0,2014-04-17 14:25:59,8727.0,2014-04-17 14:25:59,45.0,2014-04-17 14:25:59,...,2014-04-17 14:26:01,45.0,2014-04-17 14:26:01,5320.0,2014-04-17 14:26:18,5320.0,2014-04-17 14:26:47,5320.0,2014-04-17 14:26:48,198
3,303,2014-03-21 10:12:24,19.0,2014-03-21 10:12:36,303.0,2014-03-21 10:12:54,303.0,2014-03-21 10:13:01,303.0,2014-03-21 10:13:24,...,2014-03-21 10:13:36,303.0,2014-03-21 10:13:54,309.0,2014-03-21 10:14:01,303.0,2014-03-21 10:14:06,303.0,2014-03-21 10:14:24,34
4,1359,2013-12-13 09:52:28,925.0,2013-12-13 09:54:34,1240.0,2013-12-13 09:54:34,1360.0,2013-12-13 09:54:34,1344.0,2013-12-13 09:54:34,...,2013-12-13 09:54:34,1346.0,2013-12-13 09:54:34,1345.0,2013-12-13 09:54:34,1344.0,2013-12-13 09:58:19,1345.0,2013-12-13 09:58:19,601
5,11,2013-11-26 12:35:29,85.0,2013-11-26 12:35:31,52.0,2013-11-26 12:35:31,85.0,2013-11-26 12:35:32,11.0,2013-11-26 12:35:32,...,2013-11-26 12:35:32,11.0,2013-11-26 12:37:03,85.0,2013-11-26 12:37:03,10.0,2013-11-26 12:37:03,85.0,2013-11-26 12:37:04,273


In [6]:
y = train_df_400['user_id']

In [7]:
sites = ['site' + str(i) for i in range(1, 11)]
train_test_df = pd.concat([train_df_400, test_df_400])
train_test_df_sites = train_test_df[sites].fillna(0).astype('int')
train_test_df_sites.head(10)

,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10
session_id,,,,,,,,,,
1,23713,23720,23713,23713,23720,23713,23713,23713,23713,23713
2,8726,8725,665,8727,45,8725,45,5320,5320,5320
3,303,19,303,303,303,303,303,309,303,303
4,1359,925,1240,1360,1344,1359,1346,1345,1344,1345
5,11,85,52,85,11,52,11,85,10,85
6,83,1344,1240,1359,1345,85,1346,83,925,1359
7,13585,13585,13585,13585,13585,13585,13585,13585,13586,13585
8,29312,29311,29312,29312,0,0,0,0,0,0
9,21,62,18,1387,7,72,1386,1384,10,22


In [8]:
import numpy as np

def term_matrix(docs):
    """
    Returns parameters for construction sparse matrix from initial `docs` table.
    vocabulary - dictionary of terms, value as index.
    indicies - list of term indicies.
    data - list of data existence
    indptr - 
    """
    indptr = [0]
    indices = []
    data = []
    vocabulary = {}
    for d in docs:
        for term in d:
                index = vocabulary.setdefault(term, term)
                indices.append(index)
                data.append(1)
        indptr.append(len(indices))
    return data, indices, indptr

In [9]:
train_test_sparse = csr_matrix(term_matrix(
    np.array(train_test_df_sites)))[:, 1:]
train_len = train_df_400.shape[0]
X_train_sparse = train_test_sparse[:train_len]
X_test_sparse = train_test_sparse[train_len:]

In [10]:
train_df = train_test_df_sites[:train_len]
test_df = train_test_df_sites[train_len:]

In [11]:
y.head()

session_id
1    653
2    198
3     34
4    601
5    273
Name: user_id, dtype: int64

In [12]:
train_share = int(.7 * train_df.shape[0])
train_df_part = train_df.iloc[:train_share, :]
valid_df = train_df.iloc[train_share:, :]
X_train_part_sparse = X_train_sparse[:train_share, :]
X_valid_sparse = X_train_sparse[train_share:, :]

In [13]:
y_train_part = y[:train_share]
y_valid = y[train_share:]

In [14]:
logit = LogisticRegression(random_state=17, n_jobs=-1)
sgd_logit = SGDClassifier(loss='log', max_iter=3, random_state=17, n_jobs=-1)

In [15]:
%%time
sgd_logit.fit(X_train_part_sparse, y_train_part)

CPU times: user 1min 37s, sys: 33.2 s, total: 2min 11s
Wall time: 18.6 s


SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='log', max_iter=3, n_iter=None,
       n_jobs=-1, penalty='l2', power_t=0.5, random_state=17, shuffle=True,
       tol=None, verbose=0, warm_start=False)

In [16]:
from sklearn.metrics import accuracy_score

In [17]:
accuracy_score(y_valid,sgd_logit.predict(X_valid_sparse))

0.2910755315657026

In [18]:
%%time
logit.fit(X_train_part_sparse, y_train_part)

/home/maestro/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:1232: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))


CPU times: user 1h 58min 33s, sys: 2h 10min 23s, total: 4h 8min 57s
Wall time: 32min 56s


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=-1,
          penalty='l2', random_state=17, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [19]:
accuracy_score(y_valid,logit.predict(X_valid_sparse))

0.36292352018673185